## Assignment 4

For this assignment, you will parse the results of an alignment ( called a SAM file ) and compare it to
a file that contains gene annotations (GTF) . The goal is to count the number of sequences in the
SAM file that match each of the genes in the GTF file

#### **My Thoughts**
1. read in the SAM file
2. split alignments in the SAM file based on chromosomes. Do this by creating a dict where *key*: chromosome, *value*: list of start coordinates

**some lines do not have a value for chromosomes which means that sequence did not align to the genome, so we can ignore them** 

3. go through each exon in the GTF file and count the number of reads that are within the start and end coordinates. 
- since we know which chromosome the gene is in, should have to go through the start positions one at a time to see how many are within the start and end coordinate

##### *Reading in the SAM file*:

I am going to read it in, go line by line and for each line I will extract column 3 and 4, then organize it into a dictionary called **alignments**

In [ ]:
#first I need to read in the SAM file and create a dictionary
#Column 3 (Chromosome) will be the key and Column 4 (Start Pos) with the value

alignments = {} #initialize my empty dictionary

sam_file = r'C:\Users\tiffa\OneDrive\Desktop\Masters in Bioinformatics\Algorithms and Data Structures\sample.sam' 
# This is the path file to sample.sam
with open(sam_file, 'r') as file: 
#using the open method, open the sam file in reading mode
    for line in file: # go through each line in the file
        fields = line.strip().split('\t')
#split the line into fields based on the tab

        chromosome = fields[2]#the chromosome is in column 3
        start_position = int(fields[3])
#start pos is in column 4, and change the type to int

        if chromosome != '*': #if there is a value for the chromosome
            if chromosome not in alignments: 
#if the chromosome is not in the dictionary, add it
                alignments[chromosome]= []
            

            alignments[chromosome].append(start_position)
#Add the start position to the list of start positions for the chromosome

display(alignments)#for viewing purposes, looks sorect so far

I am going to sort each of the chromosome lists (with merge sort). That way when it comes time to see if it is between the start and end of an exon in a gene, it will be easier. 

In [2]:
import math
# need the math library to do floor rounding
# this merge sort algorithm is taken from the lecture notes!

def merge_sort(A, p, r):
#merge_sort is recursive
#as long as p is < r, it will recursively call on itself until it hits the base case
#then which the smaller arrays will be joined together with the merge function. 

    if p < r:
#recursively call the function as long as p is less than r
        q = math.floor((p + r) / 2)
        merge_sort(A, p, q)
        merge_sort(A, q + 1, r)
        merge(A, p, q, r)
    
    return A #return the array



def merge(A, p, q, r):
#initialize the n1 and n2 variables
    n1 = q - p + 1
    n2 = r - q
    
#initialize arrays
    L = [0] * (n1 + 1)
    R = [0] * (n2 + 1)
    
#fill in our temporary left and right arrays
    for i in range(n1):
        L[i] = A[p + i]
    for j in range(n2):
        R[j] = A[q + j + 1]
    
#float('inf) is infinity, these are our sentinels
    L[n1] = float('inf')
    R[n2] = float('inf')
    
#reset our counters for the temp arrays
    i = 0
    j = 0
    
#Compare the temp arrays to order them properly
    for k in range(p, r + 1):
        if L[i] <= R[j]:
            A[k] = L[i]
            i += 1
        else:
            A[k] = R[j]
            j += 1



In [3]:
#just an example to test my merge algorithm
#it appears to work correctly
array = [5, 2, 4, 6, 1, 3, 2, 6]
length = len(array)
merge_sort(array, 0, length-1)
print(array)

[1, 2, 2, 3, 4, 5, 6, 6]


In [ ]:
#merge sorting the alignments dictionary

sorted_alignments = {}
#initialize our sorted dictionary


for chromosomes, start_pos in alignments.items():
    sorted_alignments[chromosomes] = merge_sort(start_pos, 0, len(start_pos)-1)

display(sorted_alignments) 
#for viweing purposes, appears to have sorted my lists of start positions
#I needed to clear the outputs from this cell 
#or else the submission would be unreasonable long

##### *Reading in the GTF file*:
I am going to do the same with the GTF as the SAM. I will read it in line by line, extract the chromosome, start position and end position. Then compare it to the alignments dictionary. If the value in the dictionary is greater than the start AND less than the end, I will incrememnt a counter.
- I will need separate counters for each of the genes! (doing this with a dictionary)
- associate with gene_name
- only interested in exons

***The below code is being done without sorting the arrays!***

In [ ]:
#UNSORTED 

import re
#I will be extracting the gene name from column 9 with regex
import time
start_time = time.process_time()

#Time counting function given in the assignment

gtf_file = r'C:\Users\tiffa\OneDrive\Desktop\Masters in Bioinformatics\Algorithms and Data Structures\Arabidopsis.gtf'
#file path for thee GTF file
total_count = 0 #intialize total count for the unsorted arrays 
with open(gtf_file, 'r') as file:#open the file in reading mode
    gene_counter = {}
#initialize our counter

    for line in file: #going line by line through the file
        fields = line.strip().split('\t') 
#split the fields in the same manner as above

        if fields[2] == 'exon': # we only want information for exons!
            start = int(fields[3])#start position is column 4, modify to type int
            end = int(fields[4]) #end positions is column 5, modify to type int
            curr_chr = fields[0] #extract which chromosome we are working on
            
            match = re.search(r'gene_name "([^"]+)"', fields[8])
#extracting the gene name from the ninth column using regex
#I only want what is between quotes after gene_name

            curr_start_pos = alignments.get(curr_chr, [])
#get the list of start positions for this chromosome from the UNSORTED alignments dictionary

            if match: 
#if there is gene name in the string, then we assign it to the variable as gene_name
                gene_name = match.group(1)
# Print the gene name for testing purposes
                #print(gene_name)

            if gene_name not in gene_counter:
                gene_counter[gene_name] = 0 
#if the gene name is not in the counter dictionary, initialize the counter to 0 and add it to the dictionary

            for position in curr_start_pos:
#loop through each start position, check to see if it between the start and end position of the chromosome
                if start <= position <= end:
#if it is, then add to the counter for this gene
                    gene_counter[gene_name] +=1
                    total_count += 1

#I will be exporting the dictionary into a separate textfile since the output is so large. 
#Below is the code to write out the dictionary
outfile = r'C:\Users\tiffa\OneDrive\Desktop\Masters in Bioinformatics\Algorithms and Data Structures\Assignment 4\HW4_Unsorted_Output.txt'
#Define the outfile path

with open(outfile, 'w') as file:
    #use the open method to open the file in writing mode
    file.write("UNSORTED OUTPUT \n")
#Write my header for the file
    for gene, count in gene_counter.items():
        #go through each line of the dictionary
        file.write(f"{gene}: {count}\n")
        #write out the gene and the count

display(gene_counter) #for viewing purposes
print(time.process_time() - start_time) #time calculation.
#I needed to clear the outputs from this cell 
#or else the submission would be unreasonable long



#Count outputs follow the code in the PDF
#Time and total count is below
        

In [6]:
print(total_count)

226612


**UnSorted Array**: 
*Total Count*: 226612

*Time*: 1892.390625

**Output is in the file HW4_Unsorted_Output.txt**

***Below is the code with the sorted array!***

In [ ]:
#SORTED

#for the sorted array I am going to negate everything after the end position. 
#This should cut down on run times. 
start_time = time.process_time()
gtf_file = r'C:\Users\tiffa\OneDrive\Desktop\Masters in Bioinformatics\Algorithms and Data Structures\Arabidopsis.gtf'
sorted_total_count = 0 #initial total count for the sorted

with open(gtf_file, 'r') as file: #opening file in same manner as above
    sorted_gene_counter = {} #initialized the sorted gene counter

    for line in file:# go line by line through the file
        fields = line.strip().split('\t')#split into fields
        if fields[2] == 'exon': #only want exons
            start = int(fields[3])#start position is column 4, modify to type int
            end = int(fields[4]) #end positions is column 5, modify to type int
            curr_chr = fields[0] #extract which chromosome we are working on
            match = re.search(r'gene_name "([^"]+)"', fields[8])

            curr_start_pos = sorted_alignments.get(curr_chr, [])
#get SORTED list of start positions

            if match: 
#if there is gene name in the string, then we assign it to the variable as gene_name
                gene_name = match.group(1)
# Print the gene name for testing purposes
                #print(gene_name)

            if gene_name not in sorted_gene_counter:
                sorted_gene_counter[gene_name] = 0 
#if the gene name is not in the counter dictionary, initialize the counter to 0 and add it to the dictionary

            for position in curr_start_pos:
#loop through each start position, check to see if it between the start and end position of the chromosome
                if start <= position <= end:
#if it is, then add to the counter for this gene
                    sorted_gene_counter[gene_name] +=1   
                    sorted_total_count += 1

#if the position is greater than the end, then we will move onto the next line in the file
                if position > end:
                    break

#In the same manner as above, I will write out my dictionary to a separte output file

sorted_outfile = r'C:\Users\tiffa\OneDrive\Desktop\Masters in Bioinformatics\Algorithms and Data Structures\Assignment 4\HW4_Sorted_Output.txt'
#define my path

with open(sorted_outfile, 'w') as file:
    #open my file
    file.write('SORTED OUTPUT \n')
    #write in my header
    for gene, count in sorted_gene_counter.items():
        #go through line by line and write out the gene and count
        file.write(f"{gene}: {count}\n")

print(time.process_time() - start_time) #Time calculations
display(sorted_gene_counter)#for viewing purposes
#I needed to clear the outputs from this cell 
#or else the submission would be unreasonable long



#Count outputs follow the code in the PDF
#Time and total count is below

In [8]:
print(sorted_total_count)

226612


**Sorted Array**: 

*Time*: 586.515625

*Total Count*: 226612

**Output is in file HW4_Sorted_Output.txt**


The sorted array is faster to parse and get counts for since it does not need to go through every start position of every chromosome. In the worst case (lets say all positions for a chromosome lies between the start and end in the given GTF) it would need to go through each one- at this rate the running time would be equal to parsing through the unsorted array. Where as the unsorted array you MUST go through every start position for the chromosome and compare it to the start and end in the GTF. Even if going through the sorted array you can skip one start position for a chromosome (since it would be greater than the end of an exon) it would save you time. 

You can see from the run times themselves, 586.515625 for the sorted array and 1892.390625 for the unsorted. 